In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd
import numpy as np

In [2]:
sc = SparkContext('local')
sqlCtx = SQLContext( sc )

In [3]:
data1 = ['홍길동,1000','이순신,2000','임꺽정,3000',
         '김철수,4000','이황,5000','이이,6000']

In [5]:
data1Rdd = sc.parallelize(data1)
data1Rdd

ParallelCollectionRDD[1] at parallelize at PythonRDD.scala:195

In [6]:
data1Rdd.collect()

['홍길동,1000', '이순신,2000', '임꺽정,3000', '김철수,4000', '이황,5000', '이이,6000']

### 0. name salary 컬럼명을 가지는 데이터프레임을 만드시요.

In [35]:
dept1Rdd1 = data1Rdd.map(lambda v:v.split(',')).map(lambda v:(v[0],int(v[1])))
dept1Rdd1.collect()

[('홍길동', 1000),
 ('이순신', 2000),
 ('임꺽정', 3000),
 ('김철수', 4000),
 ('이황', 5000),
 ('이이', 6000)]

In [36]:
salaryDF1 = dept1Rdd1.toDF(['name','salary'])
salaryDF1.show()

+------+------+
|  name|salary|
+------+------+
|홍길동|  1000|
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
|  이황|  5000|
|  이이|  6000|
+------+------+



In [37]:
# 테이블 정보 확인
salaryDF1.printSchema()

root
 |-- name: string (nullable = true)
 |-- salary: long (nullable = true)



In [14]:
# 테이블 이름 지정
salaryDF1.createOrReplaceTempView('salary')

### 1. 급여가 가장높은 name과 salary를 출력하시요

In [28]:
sql_top = "select * from salary order by salary desc limit 1"
sqlDF_top = sqlCtx.sql(sql_top)
sqlDF_top.show()

+----+------+
|name|salary|
+----+------+
|이이|  6000|
+----+------+



###  2.  name, salary, tax 를 출력하시요 (tax는 급여에서 세금 3.3을 제한값

In [30]:
sql_add = "select name, salary, (salary-salary*0.033) as tax from salary"
sqlDF_add = sqlCtx.sql(sql_add)
sqlDF_add.show()

+------+------+------+
|  name|salary|   tax|
+------+------+------+
|홍길동|  1000| 967.0|
|이순신|  2000|1934.0|
|임꺽정|  3000|2901.0|
|김철수|  4000|3868.0|
|  이황|  5000|4835.0|
|  이이|  6000|5802.0|
+------+------+------+



### 3. name 에 '철'이 포함된  name,salary 를 출력하시요

In [23]:
sql = "select * from salary where name rlike '철'"
# sql = "select * from salary where name like '%철%'"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|김철수|  4000|
+------+------+



### 4. salary top 5 인 name, salary 출력하시요

In [21]:
sql = "select * from salary order by salary desc limit 5"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|  이이|  6000|
|  이황|  5000|
|김철수|  4000|
|임꺽정|  3000|
|이순신|  2000|
+------+------+



### 5. salary  2000  과 4000 사이의 데이터를 출력하시요

In [18]:
sql = "select * from salary where salary >= 2000 and salary <= 4000"
sqlDF = sqlCtx.sql(sql)
sqlDF.show()

+------+------+
|  name|salary|
+------+------+
|이순신|  2000|
|임꺽정|  3000|
|김철수|  4000|
+------+------+



In [38]:
sc.stop()